# scikit-learn scratchpad

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report
import os
from PIL import Image

'0.20.0+cu124'

In [31]:
# Custom dataset class with preprocessing
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['other', 'starmap']
        self.file_paths = []
        
        for class_name in self.classes:
            class_path = os.path.join(root_dir, class_name)
            for file in os.listdir(class_path):
                self.file_paths.append(
                    (os.path.join(class_path, file), self.classes.index(class_name))
                )

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path, label = self.file_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        # Custom crop coordinates (adjust these values as needed)
        left = 165   # starting x coordinate
        top = 1192    # starting y coordinate
        right = left + (2560 - 2117)  # ending x coordinate
        bottom = top + (1440 - 211) # ending y coordinate
        image = image.crop((left, top, right, bottom))
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create dataset and split
dataset = CustomDataset('data/screenshots', transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size]
)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model setup
model = models.resnet34(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 output classes
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}')

# Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds, target_names=dataset.classes))


C:\Users\roland\dev\fun-pytorch\sc-screenshot-classify\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10 - Loss: 0.5795
Epoch 2/10 - Loss: 1.0712
Epoch 3/10 - Loss: 0.0215
Epoch 4/10 - Loss: 0.0107
Epoch 5/10 - Loss: 0.0020
Epoch 6/10 - Loss: 0.0019
Epoch 7/10 - Loss: 0.0052
Epoch 8/10 - Loss: 0.0051
Epoch 9/10 - Loss: 0.0015
Epoch 10/10 - Loss: 0.0022
              precision    recall  f1-score   support

       other       1.00      1.00      1.00        11
     starmap       1.00      1.00      1.00         4

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15



let's save the model for later

In [32]:
torch.save(model.state_dict(), 'resnet34_state_dict_v2.pth')

Let's try to use the model now.

In [49]:
import torch
from torchvision import transforms
from PIL import Image

def load_model(model_path, device='cpu'):
    # Initialize model
    model = models.resnet34(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model.load_state_dict(torch.load(model_path, map_location=device,weights_only=False))
    model.eval()
    return model.to(device)

# Preprocessing (must match training preprocessing)
def preprocess_image(image_path):
    # transform = transforms.Compose([
    #     transforms.Resize((224, 224)),
    #     transforms.ToTensor(),
    #     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # ])
    
    img = Image.open(image_path).convert('RGB')
    # Apply the same crop as during training
    
    
    left = 165
    top = 1192
    right = left + (2560 - 2117) 
    bottom = top + (1440 - 211)
    img = img.crop((left, top, right, bottom)) 
    return transform(img).unsqueeze(0)  # Add batch dimension

# Example usage
def example_usage(image_path):
    model = load_model('resnet34_state_dict_v2.pth')
    input_tensor = preprocess_image(image_path)
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()
        print(f'Predicted class: {["other", "starmap"][predicted_class]}')


In [50]:
example_usage('E:/bin/StarCitizen/LIVE/ScreenShots/ScreenShot-2024-05-11_21-58-40-1A3.jpg')

Predicted class: other


In [51]:
example_usage('E:/bin/StarCitizen/LIVE/ScreenShots/ScreenShot-2024-05-12_00-09-07-EF4.jpg')

Predicted class: other


In [56]:
example_usage(r"E:\bin\StarCitizen\LIVE\ScreenShots\ScreenShot-2024-07-31_01-13-41-B9A.jpg")

Predicted class: other


In [52]:
example_usage(r"E:\bin\StarCitizen\LIVE\ScreenShots\ScreenShot-2024-08-13_23-34-21-1CC.jpg") # starmap

Predicted class: starmap


In [54]:
example_usage(r"E:\bin\StarCitizen\LIVE\ScreenShots\ScreenShot-2025-03-06_22-49-40-3A1.jpg")

Predicted class: starmap


In [55]:
example_usage(r"E:\bin\StarCitizen\LIVE\ScreenShots\ScreenShot-2025-03-06_23-52-52-0D2.jpg") # starmap

Predicted class: starmap
